In [1]:
import numpy as np

from scipy.constants import e, c

from scipy.constants import physical_constants

import os

In [2]:
from cpymad.madx import Madx

import sixtracklib as stl
import pysixtrack

# Versioning

## SixTrackLib

In [3]:
f = os.path.dirname(stl.__file__)
f

'/home/oeftiger/gsi/git/sixtracklib_aoeftiger/python/sixtracklib'

In [4]:
!cd $f && git log | head -4

commit f42c420da20602c9c4c08172338ed3019bb20d25
Merge: deff9373 20f49532
Author: Adrian Oeftiger <a.oeftiger@gsi.de>
Date:   Wed Jan 8 14:52:08 2020 +0100


## PySixTrack

In [5]:
f = os.path.dirname(pysixtrack.__file__)
f

'/home/oeftiger/gsi/git/pysixtrack/pysixtrack'

In [6]:
!cd $f && git log | head -4

commit a1c73d9578a20e404dff54e1330aebe77eb763d5
Merge: 1d8f5db 4ce2927
Author: Riccardo De Maria <riccardodemaria@gmail.com>
Date:   Wed Jan 8 17:48:29 2020 +0100


# Parameters

In [7]:
nmass = physical_constants['atomic mass constant energy equivalent in MeV'][0] * 1e-3
# nmass = 0.931494061 # MAD-X value

In [8]:
A = 238
Q = 28

Ekin_per_nucleon = 0.2e9 # in eV

###

mass = A * nmass * 1e9 * e / c**2 # in kg
charge = Q * e # in Coul

Ekin = Ekin_per_nucleon * A
p0c = np.sqrt(Ekin**2 + 2*Ekin*mass/e * c**2) # in eV

Etot = np.sqrt(p0c**2 + (mass/e)**2 * c**4) * 1e-9 # in GeV
p0 = p0c / c * e # in SI units
gamma = np.sqrt(1 + (p0 / (mass * c))**2)
beta = np.sqrt(1 - gamma**-2)

## MAD-X setup

In [9]:
madx = Madx()
madx.options.echo = False
madx.options.warn = False


  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.05.01  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2019.06.07               +
  + Execution date: 2020.01.14 19:01:59      +
  ++++++++++++++++++++++++++++++++++++++++++++


In [10]:
madx.input('''SET, format="22.14e";''')

True

In [11]:
madx.input('''
qf: multipole, knl={-0.05817764173};

testing: sequence, l = 1;
qf, at = 0.5;
endsequence;
''')

True

In [12]:
madx.command.beam(
    particle='ion', mass=A*nmass, charge=Q, energy=Etot)

True

In [13]:
madx.use(sequence='testing')

In [14]:
twiss_no_error = madx.twiss(betx=1, bety=1, dx=0, dy=0);

enter Twiss module

++++++ table: summ

                length                 orbit5                   alfa                gammatr 
  1.00000000000000e+00  -0.00000000000000e+00   0.00000000000000e+00   0.00000000000000e+00 

                    q1                    dq1                betxmax                  dxmax 
  1.25000000000000e-01   0.00000000000000e+00   2.00000000000000e+00   5.12406983876715e-02 

                 dxrms                 xcomax                 xcorms                     q2 
  3.24074631605519e-02   0.00000000000000e+00   0.00000000000000e+00   1.25000000000000e-01 

                   dq2                betymax                  dymax                  dyrms 
  0.00000000000000e+00   2.00000000000000e+00   0.00000000000000e+00   0.00000000000000e+00 

                ycomax                 ycorms                 deltap                synch_1 
  0.00000000000000e+00   0.00000000000000e+00   0.00000000000000e+00   0.00000000000000e+00 

               synch_2   

In [15]:
q1_no_error = twiss_no_error.summary.q1

In [16]:
madx.select(flag='error', pattern='qf')

True

In [17]:
# madx.command.efcomp(
#     dkn=[3.367586806456914e-06, 6.784039963878577e-05, -0.024481002280817638, 0.009464767004443361, -294.58727168196225, -142.28516714931774, -486509.91834112257,],
#     dks=[0.0, 8.28311433972563e-05, -0.002797094898967373, -0.0055152719717576825, -65.5798575457456, -336.91869644950197, -7419.614743085283,],
# )

In [18]:
madx.command.ealign(dpsi=np.pi/2) #-0.00015308942422657729)

Assigned alignment errors to 1 elements


True

In [19]:
# qf = madx.sequence.testing.elements[1] 
### --> only "elements" does not work!! need "expanded_elements"
qf = madx.sequence.testing.expanded_elements[2]
qf

qf: multipole, at=0.5, knl={-0.05817764173};

In [20]:
# qf.field_errors.dkn, qf.field_errors.dks

In [21]:
qf.align_errors.dx

0.0

In [22]:
qf.align_errors.dy

0.0

In [23]:
qf.align_errors.dpsi

1.5707963267948966

In [24]:
twiss_with_error = madx.twiss(betx=1, bety=1, dx=0, dy=0);

enter Twiss module

++++++ table: summ

                length                 orbit5                   alfa                gammatr 
  1.00000000000000e+00  -0.00000000000000e+00   0.00000000000000e+00   0.00000000000000e+00 

                    q1                    dq1                betxmax                  dxmax 
  1.25134860520863e-01   0.00000000000000e+00   1.99662204605264e+00   0.00000000000000e+00 

                 dxrms                 xcomax                 xcorms                     q2 
  0.00000000000000e+00   2.91216837334142e-02   1.84181699793329e-02   1.25134860520863e-01 

                   dq2                betymax                  dymax                  dyrms 
  0.00000000000000e+00   1.99662204605264e+00   5.13275364556708e-02   3.24623843770492e-02 

                ycomax                 ycorms                 deltap                synch_1 
  2.91710365102473e-02   1.84493834160622e-02   0.00000000000000e+00   0.00000000000000e+00 

               synch_2   

In [25]:
q1_with_error = twiss_with_error.summary.q1

$\implies$ **The tune has changed with the additional quadrupole error:**

In [26]:
1 - q1_no_error / q1_with_error

0.0010777214303165916

## MAD-X tracking

In [27]:
madx.input('''
TRACK, onepass, onetable, file=output.;

!START, x=1e-03, px=1e-06, y=-1e-03, py=-5e-07, t=0.1, pt=3e-03;
!START, x=0e-03, px=0e-06, y=0e-03, py=0e-07, t=0.1, pt=3e-03;
START, x=0e-03, px=0e-06, y=0e-03, py=0e-07, t=0, pt=0e-03;

RUN, turns=1;

ENDTRACK;
''')

enter TRACK module
one pass is on

++++++ table: tracksumm

    number       turn                      x                     px 
         1          0   0.00000000000000e+00   0.00000000000000e+00 
         1          1  -2.92211477491618e-02  -5.82433674668284e-02 

                     y                     py                      t                     pt 
  0.00000000000000e+00   0.00000000000000e+00   0.00000000000000e+00   0.00000000000000e+00 
  2.92706690885483e-02   5.83420730204945e-02  -3.00821197573364e-03   0.00000000000000e+00 

                     s                      e 
  0.00000000000000e+00   0.00000000000000e+00 
  1.00000000000000e+00   0.00000000000000e+00 
exit TRACK module



True

In [28]:
headers = list(np.genfromtxt(
    "output.one", skip_header=51, max_rows=1, dtype=str)[1:])

initial_distribution_madx = np.genfromtxt(
    "output.one", skip_header=54, max_rows=1, dtype=np.float64)

final_distribution_madx = np.genfromtxt(
    "output.one", skip_header=55, max_rows=1, dtype=np.float64)

In [29]:
def get_betai(PT, mass=mass, p0=p0):
    restmass = mass * c**2
    restmass_sq = restmass**2
    E0 = np.sqrt((p0 * c)**2 + restmass_sq)
    
    E = E0 + PT * p0 * c
    gammai = E / restmass
    betai = np.sqrt(1 - 1. / (gammai * gammai))
    return betai

In [30]:
pt_madx = final_distribution_madx[headers.index('PT')]
betai = get_betai(pt_madx)

x_madx = final_distribution_madx[headers.index('X')]
px_madx = final_distribution_madx[headers.index('PX')]
y_madx = final_distribution_madx[headers.index('Y')]
py_madx = final_distribution_madx[headers.index('PY')]
zeta_madx = final_distribution_madx[headers.index('T')] * betai
delta_madx = np.sqrt(1 + 2 * pt_madx / beta + pt_madx**2) - 1

## SixTrackLib setup

In [31]:
madx.esave(file='errors')
madx.command.readtable(file='errors', table='errors')
errors = madx.table.errors

Want to make named table: errors


In [32]:
# PySixTrack, lattice transfer and preparation!

pysixtrack_elements = pysixtrack.Line.from_madx_sequence(
    madx.sequence.testing, exact_drift=True,
)

pysixtrack_elements.apply_madx_errors(errors)

pysixtrack_elements.remove_zero_length_drifts(inplace=True);
pysixtrack_elements.merge_consecutive_drifts(inplace=True);

The PySixTrack multipole has a quadrupole component consisting of the sum of the MAD-X original quadrupole component (0.3) + the error (0.01):

In [33]:
pysixtrack_elements.elements[2].knl

[-0.05817764173]

In [34]:
pysixtrack_elements.elements[1].angle

90.0000000000002

In [35]:
pysixtrack_elements.elements[2].hxl

-0.05817764173

In [36]:
elements = stl.Elements.from_line(pysixtrack_elements)
elements.BeamMonitor(num_stores=1);

In [37]:
pysixtrack_particles = pysixtrack.Particles.from_madx_track(madx)

In [38]:
# necessary to transfer to SixTrackLib with this version of PySixTrack
pysixtrack_particles.state = np.array([1, 0])
pysixtrack_particles.elemid = 0

pysixtrack_particles.remove_lost_particles()

In [39]:
particles = stl.ParticlesSet().Particles(num_particles=1)

particles.from_pysixtrack(pysixtrack_particles, particle_index=0)

In [40]:
trackjob = stl.TrackJob(elements, particles) #, device="opencl:0.0")

## SixTrackLib tracking

In [41]:
trackjob.track_until(1)
trackjob.collect()

In [42]:
x = trackjob.output.particles[0].x[0]
px = trackjob.output.particles[0].px[0]
y = trackjob.output.particles[0].y[0]
py = trackjob.output.particles[0].py[0]
zeta = trackjob.output.particles[0].zeta[0]
delta = trackjob.output.particles[0].delta[0]

In [43]:
x, px, y, py, zeta, delta

(0.0, -0.0, 0.0, 0.0, 0.0, 0.0)

## PySixTrack tracking

In [44]:
srot = pysixtrack_elements.elements[1]
multip = pysixtrack_elements.elements[2]
hxl = multip.hxl
multip.hyl = hxl * np.sin(srot.angle * np.pi / 180.)
multip.hxl = hxl * np.cos(srot.angle * np.pi / 180.)
multip

Multipole(knl=[-0.05817764173], ksl=[0.0], hxl=2.0312615043819654e-16, hyl=-0.05817764173, length=0.0)

In [45]:
pysixtrack_particles

        mass0   = 221695594.70520002
        p0c     = 152876357.28239682
        energy0 = 269295594.7052
        beta0   = 0.5676897813711055
        gamma0  = 1.2147088220823519
        s       = [0.]
        x       = [0.]
        px      = [0.]
        y       = [0.]
        py      = [0.]
        zeta    = [0.]
        delta   = [0.]
        ptau    = [0.]
        mratio  = 1.0
        qratio  = 1.0
        chi     = 1.0

In [46]:
pysixtrack_elements.track(pysixtrack_particles)

In [47]:
pysixtrack_particles

        mass0   = 221695594.70520002
        p0c     = 152876357.28239682
        energy0 = 269295594.7052
        beta0   = 0.5676897813711055
        gamma0  = 1.2147088220823519
        s       = [1.]
        x       = [-0.02918778]
        px      = [-0.05817764]
        y       = [0.02918778]
        py      = [0.05817764]
        zeta    = [-0.00170096]
        delta   = [0.]
        ptau    = [0.]
        mratio  = 1.0
        qratio  = 1.0
        chi     = 1.0

In [48]:
x = pysixtrack_particles.x[0]
px = pysixtrack_particles.px[0]
y = pysixtrack_particles.y[0]
py = pysixtrack_particles.py[0]
zeta = pysixtrack_particles.zeta[0]
delta = pysixtrack_particles.delta[0]

# evaluation SixTrackLib vs. MAD-X:

In [49]:
print ('x error in %: {}'.format(100 * (x - x_madx) / x))

x error in %: -0.11432547565039203


In [50]:
print ('px error in %: {}'.format(100 * (px - px_madx) / px))

px error in %: -0.11297421977540519


In [51]:
print ('y error in %: {}'.format(100 * (y - y_madx) / y))

y error in %: -0.2839901148322105


In [52]:
print ('py error in %: {}'.format(100 * (py - py_madx) / py))

py error in %: -0.2826365689720147


In [53]:
print ('zeta error in %: {}'.format(100 * (zeta - zeta_madx) / zeta))

zeta error in %: -0.3981054912551291


In [54]:
print ('delta error in %: {}'.format(100 * (delta - delta_madx) / delta))

delta error in %: nan


/home/oeftiger/anaconda3/envs/stl_martin/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in double_scalars
  """Entry point for launching an IPython kernel.


In [55]:
delta == delta_madx

True

# Alternative by rotating k0l

In [73]:
# PySixTrack, lattice transfer and preparation!

pysixtrack_elements = pysixtrack.Line.from_madx_sequence(
    madx.sequence.testing, exact_drift=True,
)

# pysixtrack_elements.apply_madx_errors(errors)

pysixtrack_elements.remove_zero_length_drifts(inplace=True);
pysixtrack_elements.merge_consecutive_drifts(inplace=True);

In [74]:
multip = pysixtrack_elements.elements[1]
k0l = multip.knl[0]
multip.ksl[0] = k0l * np.sin(-srot.angle * np.pi / 180.)
multip.knl[0] = k0l * np.cos(srot.angle * np.pi / 180.)
multip

Multipole(knl=[2.0312615043819654e-16], ksl=[0.05817764173], hxl=-0.05817764173, hyl=0.0, length=0.0)

In [75]:
pysixtrack_particles = pysixtrack.Particles.from_madx_track(madx)

In [76]:
# necessary to transfer to SixTrackLib with this version of PySixTrack
pysixtrack_particles.state = np.array([1, 0])
pysixtrack_particles.elemid = 0

pysixtrack_particles.remove_lost_particles()

In [77]:
pysixtrack_particles

        mass0   = 221695594.70520002
        p0c     = 152876357.28239682
        energy0 = 269295594.7052
        beta0   = 0.5676897813711055
        gamma0  = 1.2147088220823519
        s       = [0.]
        x       = [0.]
        px      = [0.]
        y       = [0.]
        py      = [0.]
        zeta    = [0.]
        delta   = [0.]
        ptau    = [0.]
        mratio  = 1.0
        qratio  = 1.0
        chi     = 1.0

In [78]:
pysixtrack_elements.track(pysixtrack_particles)

In [79]:
pysixtrack_particles

        mass0   = 221695594.70520002
        p0c     = 152876357.28239682
        energy0 = 269295594.7052
        beta0   = 0.5676897813711055
        gamma0  = 1.2147088220823519
        s       = [1.]
        x       = [-0.02918778]
        px      = [-0.05817764]
        y       = [0.02918778]
        py      = [0.05817764]
        zeta    = [-0.00170096]
        delta   = [0.]
        ptau    = [0.]
        mratio  = 1.0
        qratio  = 1.0
        chi     = 1.0

In [80]:
x = pysixtrack_particles.x[0]
px = pysixtrack_particles.px[0]
y = pysixtrack_particles.y[0]
py = pysixtrack_particles.py[0]
zeta = pysixtrack_particles.zeta[0]
delta = pysixtrack_particles.delta[0]

# evaluation SixTrackLib vs. MAD-X:

In [81]:
print ('x error in %: {}'.format(100 * (x - x_madx) / x))

x error in %: -0.11432547565039203


In [82]:
print ('px error in %: {}'.format(100 * (px - px_madx) / px))

px error in %: -0.11297421977540519


In [83]:
print ('y error in %: {}'.format(100 * (y - y_madx) / y))

y error in %: -0.2839901148322105


In [84]:
print ('py error in %: {}'.format(100 * (py - py_madx) / py))

py error in %: -0.2826365689720147


In [85]:
print ('zeta error in %: {}'.format(100 * (zeta - zeta_madx) / zeta))

zeta error in %: -0.3981054912551291


In [86]:
print ('delta error in %: {}'.format(100 * (delta - delta_madx) / delta))

delta error in %: nan


/home/oeftiger/anaconda3/envs/stl_martin/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in double_scalars
  """Entry point for launching an IPython kernel.


In [87]:
delta == delta_madx

True